# Transfer Learning with tensorflow Part 1: Feature Extraction

Transfer learning is leveraging a working model's architecture and learned patterns for our own problem.
1. Can leverage an existing neural network architecture proven to work on problems similar to our own. 
2. Can leverage a working neural network architecture which has already learned patterns on similar data to our own,then we can adapt those patterns to our own data

## Downloading and becoming one with the data

In [1]:
# Get data(10% of 10 food classes from Food101)
import zipfile

# Unzip the downloaded file
zip_ref=zipfile.ZipFile("10_food_classes_10_percent.zip","r")
zip_ref.extractall()
zip_ref.close()

In [2]:
# How many images in each folder?
import os

# Walk through 10 percent data directory and list number of files 
for dirpath,dirnames,filesnames in os.walk("10_food_classes_10_percent"):
    print(f"There are {len(dirnames)} directories and {len(filesnames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent\test'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent

## Creating data loaders(preparing the data)

We'll use the `ImageDataGenerator` class to load in our images in batches

In [3]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE=(224,224)
BATCH_SIZE=32

train_dir="10_food_classes_10_percent/train/"
test_dir="10_food_classes_10_percent/test/"

train_datagen=ImageDataGenerator(rescale=1/225.)
test_datagen=ImageDataGenerator(rescale=1/225.)

print("Training Images")
train_data_10_percent=train_datagen.flow_from_directory(train_dir,
                                                        target_size=IMAGE_SHAPE,
                                                        batch_size=BATCH_SIZE,
                                                        class_mode="categorical")
print("Testing images")
test_data_10_percent=test_datagen.flow_from_directory(test_dir,
                                                      target_size=IMAGE_SHAPE,
                                                      batch_size=BATCH_SIZE,
                                                      class_mode="categorical")

Training Images
Found 750 images belonging to 10 classes.
Testing images
Found 2500 images belonging to 10 classes.


## Setting up callbacks(things to run whilst our model trains)

Callbacks are extra functionality you can add to your models to be performed during or after training.Some of the modet popular callbacks:

* Tracking experiments with the TensorBoard callback
* Model checkpoint with the ModelCheckpoint callback
* Stopping a model from training (before it trains too long and overfits) with the EarlyStopping callback


In [4]:
import tensorflow as tf

In [15]:
# Create Tensorboard callback (functionized beacuse we need to create a new one for each model)
import datetime

def create_tensorborad_callback(dir_name,experiment_name):
    log_dir=dir_name+"/"+experiment_name+"/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"Saving TensorBoard log files to :{log_dir}")
    return tensorboard_callback

## Creating models using Tensorflow Hub

In the past we've used tensorflow to create our models layer by layers from scratch.

Now we'regoing to do a similar process,except the majority of our model's layers are going to come from tensorflow Hub.
We can access pretrained models on: [Kaggle is now TensorFlow Hub](https://www.tensorflow.org/hub)

Browsing the **TensorFlow Hub(NOW KAGGLE)** and sorting for image classification ,we found the follwoing feature vector model link :  https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1

In [16]:
# Let's compare the follwoing two models
resnet_url = "https://tfhub.dev/tensorflow/resnet_50/feature_vector/1"

efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [17]:
# Import dependencies
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [18]:
# Let's make a create_model() function to create a model from a URL
def create_model(model_url,num_classes=10):
    """
    Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.
    
    Args:
    model_url(str):A TensorFlow Hub feature extraction URL.
    num_classes(int): Number of output neurons in the output layer,
    should be equal to number of target classes,default 10.

    Returns:
    An uncompiled Keras Sequential model with model_url as feature extractor layer and Dense output
    layer with num_classes output neurons.
    """
    # Download the pretrained model and save it as Keras layer
    feature_extractor_layer=hub.KerasLayer(model_url,
                                           trainable=False,
                                           name="feature_extractions_layer",
                                            input_shape=IMAGE_SHAPE+(3,)) # freeze already learned patterns
    # Create our model
    model=tf.keras.Sequential([
        feature_extractor_layer,
        layers.Dense(num_classes,activation="softmax",name="output_layer")
    ])
    return model

### Creating and testing ResNet Tensorflow Hub Feature Extraction model

In [19]:
# Create ResNet model
resnet_model=create_model(resnet_url,num_classes=train_data_10_percent.num_classes)

# Compile 
resnet_model.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [20]:
resnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extractions_layer (  (None, 2048)             23561152  
 KerasLayer)                                                     
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23,581,642
Trainable params: 20,490
Non-trainable params: 23,561,152
_________________________________________________________________


In [21]:
# Let's fit our ResNet model to the data(10 percent of 10 classes)
resnet_history=resnet_model.fit(train_data_10_percent
                                ,epochs=5,
                                steps_per_epoch=len(train_data_10_percent),
                                validation_data=test_data_10_percent,
                                callbacks=[create_tensorborad_callback(dir_name="tensorflow_hub",
                                                                       experiment_name="resnet50v2")],
                                validation_steps=len(test_data_10_percent))

Saving TensorBoard log files to :tensorflow_hub/resnet50v2/20250530-001535
Epoch 1/5
24/24 [==============================] - 17s 507ms/step - loss: 1.9805 - accuracy: 0.3213 - val_loss: 1.1269 - val_accuracy: 0.6332
Epoch 2/5
24/24 [==============================] - 11s 450ms/step - loss: 0.8591 - accuracy: 0.7347 - val_loss: 0.8023 - val_accuracy: 0.7480
Epoch 3/5
24/24 [==============================] - 11s 450ms/step - loss: 0.5576 - accuracy: 0.8413 - val_loss: 0.6983 - val_accuracy: 0.7752
Epoch 4/5
24/24 [==============================] - 11s 451ms/step - loss: 0.4580 - accuracy: 0.8760 - val_loss: 0.5896 - val_accuracy: 0.8104
Epoch 5/5
24/24 [==============================] - 10s 448ms/step - loss: 0.3357 - accuracy: 0.9213 - val_loss: 0.5976 - val_accuracy: 0.8012


Wow!!!

That is Incredible Our transfer learning feature extractor model out performed ALL of the previous models we built by hand..(substantially) and in a quicker training time AND with only 10 % of the training examples

In [22]:
# Let's create a function to plot our loss curves